In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('/Users/aligebesce/PycharmProjects/wikimedia-mt-analysis/src/terimlervswiki/terms/terimler_org/terimler_org_all.xlsx')
df = df[df['number_of_en'] != 0]
en_terms = df['en'].values
en_terms = [str(en) for en in en_terms]

all_terms = []
for en in en_terms:
    if ';' in en:
        en = en.split(';')
        all_terms.extend(en)
    else:
        all_terms.append(en)

# trim whitespaces at the beginning and end
all_terms = [en.strip() for en in all_terms]

# lower all the letters
all_terms = [en.lower() for en in all_terms]

# normalize all: 
def normalize_term(term):
    """Normalize special characters in a term and return its URL part."""
    replacements = {"â": "a", 'ç': 'c', 'ğ': 'g', 'ı': 'i', 'ö': 'o', 'ş': 's', 'ü': 'u', 'Ç': 'C', 'Ğ': 'G', 'İ': 'I',
                    'Ö': 'O', 'Ş': 'S', 'Ü': 'U', "'": "", "’": "", }
    # replace all punctiation with empty string
    term = term.translate(str.maketrans('', '', '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')).lower()
    return term.translate(str.maketrans(replacements))

all_terms = [normalize_term(en) for en in all_terms]
all_terms = [term for term in all_terms if term != ' ' and term != '']
all_terms = set(all_terms)

# read the translation pairs
translations_pairs = pd.read_excel('/Users/aligebesce/PycharmProjects/wikimedia-mt-analysis/src/translationpairs/mt_and_target_present.xlsx')
# reverse the rows of the dataframe
translations_pairs = translations_pairs.iloc[::-1]
# drop nan values
translations_pairs = translations_pairs.dropna(subset=['source'])
# add new column source_normalized
translations_pairs['source_normalized'] = translations_pairs['source'].apply(normalize_term)

In [3]:
def find_exact_multiword_matching_terms(source_normalized, all_terms):
    """
    Check if any term from all_terms, which may contain multiple words, is contained within the source_normalized text.
    """
    # Initialize an empty list to store matching terms
    matching_terms = []
    # Iterate through each term in all_terms
    for term in all_terms:
        # Check if the term is fully contained in the source_normalized string
        if ' ' + term + ' ' in ' ' + source_normalized + ' ':  # Adding spaces to catch full terms only
            matching_terms.append(term)
    return matching_terms



# Applying the function to create the new column 'matching_terms'
translations_pairs['matching_terms'] = translations_pairs['source_normalized'].apply(find_exact_multiword_matching_terms, args=(all_terms,))


In [4]:
# save the dataframe to a new excel file
translations_pairs.to_excel('/Users/aligebesce/PycharmProjects/wikimedia-mt-analysis/src/translationpairs/mt_and_target_present_with_matching_terms.xlsx', index=False)

In [5]:
# read the new excel file
translations_pairs = pd.read_excel('/Users/aligebesce/PycharmProjects/wikimedia-mt-analysis/src/translationpairs/mt_and_target_present_with_matching_terms.xlsx')
# get only if is_scientific is not null or empty
translations_pairs = translations_pairs[translations_pairs['is_scientific'].notnull()]
# create new column contains_term which is 0 if matching_terms is  "[]" and 1 if not
translations_pairs['contains_term'] = translations_pairs['matching_terms'].apply(lambda x: 0 if x == "[]" else 1)

In [6]:
# save the dataframe to a new excel file
translations_pairs.to_excel('/Users/aligebesce/PycharmProjects/wikimedia-mt-analysis/src/translationpairs/mt_and_target_present_with_matching_terms_scientific.xlsx', index=False)

In [7]:
# read the new excel file
df = pd.read_excel('/Users/aligebesce/PycharmProjects/wikimedia-mt-analysis/src/translationpairs/example.xlsx')


In [8]:
# Get only if contains_term is 1
df = df[df['contains_term'] == 1]

In [9]:
# save the dataframe to a new excel file
df.to_excel('/Users/aligebesce/PycharmProjects/wikimedia-mt-analysis/src/translationpairs/example_contains_term.xlsx', index=False)